In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import files
import io
uploaded = files.upload()
df = pd.read_excel(io.BytesIO(uploaded['Dataset.xlsx']))

Saving Dataset.xlsx to Dataset.xlsx


In [17]:
class Regression:
    def __init__(self, learning_rate, iteration, regularization):
       
        self.m = None
        self.n = None
        self.w = None
        self.b = None
        self.regularization = regularization 
        self.lr = learning_rate
        self.it = iteration

    def cost_function(self, y, y_pred):
        
        return (1 / (2*self.m)) * np.sum(np.square(y_pred - y)) + self.regularization(self.w)
    
    def hypothesis(self, weights, bias, X):
        
        return np.dot(X, weights) 

    def train(self, X, y):
        
        print("X", X.shape)
        X1 = np.insert(X, 0, 1, axis=1)
        #np.insert(X,0,1,axis=1,inplace=True)
        print("X1", X1.shape)


      
        try:
            y.shape[1]
        except IndexError as e:
            # we need to change it to the 1 D array, not a list.
            print("ERROR: Target array should be a one dimentional array not a list"
                  "----> here the target value not in the shape of (n,1). \nShape ({shape_y_0},1) and {shape_y} not match"
                  .format(shape_y_0 = y.shape[0] , shape_y = y.shape))
            return 
        
        self.m = X1.shape[0]
        self.n = X1.shape[1]
        self.w = np.zeros((self.n , 1))
        self.b = 0

        for it in range(1, self.it+1):
            
            y_pred = self.hypothesis(self.w, self.b, X1)
            cost = self.cost_function(y, y_pred)
            dw = (1/self.m) * np.dot(X1.T, (y_pred - y)) + self.regularization.derivation(self.w)
            self.w = self.w - self.lr * dw
            if it % 10 == 0:
                print("The Cost function for the iteration {}----->{} :)".format(it, cost))
    def predict(self, test_X):
        
        test_X = np.insert(test_X, 0, 1, axis=1)

        y_pred = self.hypothesis(self.w, self.b, test_X)
        return y_pred

In [19]:
class l1_regularization:
    def __init__(self, lamda):
        self.lamda = lamda

    def __call__(self, weights):
        return self.lamda * np.sum(np.abs(weights))
    
    def derivation(self, weights):
        return self.lamda * np.sign(weights)

In [20]:
class LassoRegression(Regression):
   
    def __init__(self, lamda, learning_rate, iteration):
       
        self.regularization = l1_regularization(lamda)
        super(LassoRegression, self).__init__(learning_rate, iteration, self.regularization)

    def train(self, X, y):
       
        return super(LassoRegression, self).train(X, y)
    def predict(self, test_X):
     
        return super(LassoRegression, self).predict(test_X)

In [21]:
fileName = "Dataset.xlsx"
df_Labels = pd.read_excel(fileName, sheet_name="Plant_1_Data")
df_Labels = df_Labels.drop(['PLANT_ID'], axis=1)
df_Labels["DATE_TIME"] = pd.to_datetime(df_Labels.DATE_TIME)
df_Features = pd.read_excel(fileName, sheet_name="Plant_1_Sensor_Data")
df_Features = df_Features.drop(['PLANT_ID','SOURCE_KEY'], axis=1)
df_Features['DATE_TIME'] = pd.to_datetime(df_Features.DATE_TIME)

In [22]:
df_Learn = pd.merge(df_Features, df_Labels, how='inner', left_on = ['DATE_TIME'], right_on = ['DATE_TIME'])

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


X = df_Learn.drop(['DC_POWER', 'AC_POWER', 'DAILY_YIELD', 'TOTAL_YIELD', 'SOURCE_KEY'], axis = 1)
X['hour'] = X['DATE_TIME'].dt.hour
X['min'] = X['DATE_TIME'].dt.minute
X = X.drop(['DATE_TIME'], axis = 1)
y = df_Learn['DC_POWER']
display(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
cols_to_scale = ['AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION']
scaler = StandardScaler()
scaler.fit(X_train[cols_to_scale])
X_train[cols_to_scale] = scaler.transform(X_train[cols_to_scale])
X_test[cols_to_scale] = scaler.transform(X_test[cols_to_scale])
display(X_train)

,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,hour,min
0,25.184316,22.857507,0.0,0,0
1,25.184316,22.857507,0.0,0,0
2,25.184316,22.857507,0.0,0,0
3,25.184316,22.857507,0.0,0,0
4,25.184316,22.857507,0.0,0,0
...,...,...,...,...,...
45675,21.909288,20.427972,0.0,23,45
45676,21.909288,20.427972,0.0,23,45
45677,21.909288,20.427972,0.0,23,45
45678,21.909288,20.427972,0.0,23,45


,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,hour,min
31161,-0.670111,-0.793333,-0.772531,1,45
43720,-0.883306,-0.822172,-0.772531,1,0
14303,1.518744,1.818756,1.887604,13,0
21089,-0.240173,-0.540894,-0.772531,21,30
20303,1.794998,2.120042,2.383081,12,30
...,...,...,...,...,...
11284,1.576698,1.931014,2.173641,13,15
44732,0.748897,0.922154,0.825447,13,0
38158,0.123579,0.902551,1.139332,10,0
860,0.763700,1.636051,1.416952,10,0


In [24]:
!pip install feature_engine
from feature_engine.creation import CyclicalFeatures
cyclical = CyclicalFeatures(variables=['hour', 'min'], drop_original=True)
display(X_train)
cyclical.fit(X_train)
X_train = cyclical.transform(X_train)
X_test = cyclical.transform(X_test)

X_train

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,hour,min
31161,-0.670111,-0.793333,-0.772531,1,45
43720,-0.883306,-0.822172,-0.772531,1,0
14303,1.518744,1.818756,1.887604,13,0
21089,-0.240173,-0.540894,-0.772531,21,30
20303,1.794998,2.120042,2.383081,12,30
...,...,...,...,...,...
11284,1.576698,1.931014,2.173641,13,15
44732,0.748897,0.922154,0.825447,13,0
38158,0.123579,0.902551,1.139332,10,0
860,0.763700,1.636051,1.416952,10,0


,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,hour_sin,hour_cos,min_sin,min_cos
31161,-0.670111,-0.793333,-0.772531,0.269797,0.962917,-2.449294e-16,1.0
43720,-0.883306,-0.822172,-0.772531,0.269797,0.962917,0.000000e+00,1.0
14303,1.518744,1.818756,1.887604,-0.398401,-0.917211,0.000000e+00,1.0
21089,-0.240173,-0.540894,-0.772531,-0.519584,0.854419,-8.660254e-01,-0.5
20303,1.794998,2.120042,2.383081,-0.136167,-0.990686,-8.660254e-01,-0.5
...,...,...,...,...,...,...,...
11284,1.576698,1.931014,2.173641,-0.398401,-0.917211,8.660254e-01,-0.5
44732,0.748897,0.922154,0.825447,-0.398401,-0.917211,0.000000e+00,1.0
38158,0.123579,0.902551,1.139332,0.398401,-0.917211,0.000000e+00,1.0
860,0.763700,1.636051,1.416952,0.398401,-0.917211,0.000000e+00,1.0


In [25]:
#check shape
X_arr = np.array(X_train)
Y_arr = np.array(y_train)
print(X_arr.shape, Y_arr.shape)

(41112, 7) (41112,)


In [26]:
display(y_train)

31161        0.000000
43720        0.000000
14303     9309.142857
21089        0.000000
20303    12619.375000
             ...     
11284    12316.000000
44732     6719.142857
38158     8921.714286
860       6008.375000
15795     7756.250000
Name: DC_POWER, Length: 41112, dtype: float64

In [27]:
param = {
    "lamda" : 0.1,
    "learning_rate" : 0.1,
    "iteration" : 100
}
print("="*100)
linear_reg = LassoRegression(**param)

X_train.reset_index(drop=True, inplace=True)
# Train the model.
linear_reg.train(X_train,y_train)

# Predict the values.
y_pred = linear_reg.predict(X_test)

#Root mean square error.
score = r2_score(y, y_pred)
print("The r2_score of the trained model", score)

X (41112, 7)


ValueError: ignored

In [29]:
X_train.head()

,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,hour_sin,hour_cos,min_sin,min_cos
31161,-0.670111,-0.793333,-0.772531,0.269797,0.962917,-2.449294e-16,1.0
43720,-0.883306,-0.822172,-0.772531,0.269797,0.962917,0.000000e+00,1.0
14303,1.518744,1.818756,1.887604,-0.398401,-0.917211,0.000000e+00,1.0
21089,-0.240173,-0.540894,-0.772531,-0.519584,0.854419,-8.660254e-01,-0.5
20303,1.794998,2.120042,2.383081,-0.136167,-0.990686,-8.660254e-01,-0.5
